In [1]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download a pre-trained model
model_path = 'cased_bert_base'
if not exists(model_path):
  download_from_google_drive('1MOZUKppfX45BEh7nxQ5AvzK-8wIUITr8', '%s.zip' % model_path)
  zip_file = model_path+".zip"
  !unzip $zip_file
  sys.path.append(model_path)

In [2]:
import os
import sys
from os.path import exists, join, basename, splitext

project_path = 'mongolian-bert'
  
sys.path.append(project_path)

if not exists(project_path):
  print("checking out")
  !git clone -q --recursive https://github.com/tugstugi/mongolian-bert.git

In [3]:
!pip install -r ./mongolian-bert/requirements.txt

In [4]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

In [5]:
from tokenization_sentencepiece import FullTokenizer
tokenizer_model_path = model_path
tokenizer  = FullTokenizer(model_file=join(tokenizer_model_path, 'mn_cased.model'), vocab_file=join(tokenizer_model_path, 'mn_cased.vocab'), do_lower_case=False)

Loaded a trained SentencePiece model.


In [7]:
LEN                 = 512
BERT_PRETRAINED_DIR = "./cased_bert_base"
DATASET             = 'eduge'
OUTPUT_DIR          = './{}_{}_cased'.format(DATASET, LEN)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
print(OUTPUT_DIR)

./eduge_512_cased


In [8]:
sys.path.append("./mongolian-bert/bert")
import modeling
import optimization
import run_classifier
import tokenization

In [9]:
MODEL_CHECKPOINT = "model.ckpt-4000000"
VOCAB_FILE       = os.path.join(model_path, 'mn_cased.vocab')
CONFIG_FILE      = os.path.join(model_path, 'bert_config.json')
INIT_CHECKPOINT  = os.path.join(BERT_PRETRAINED_DIR, MODEL_CHECKPOINT)
DO_LOWER_CASE    = False

In [ ]:
import os
if not os.path.exists("eduge.csv.gz"):
  !wget https://github.com/tugstugi/mongolian-nlp/raw/master/datasets/eduge.csv.gz
  !gunzip eduge.csv.gz

In [10]:
!pip install pandas scikit-learn

In [11]:
import pandas as pd
df = pd.read_csv("eduge.csv")
df = df.rename(columns=lambda x: x.strip())
df.columns.values
labels = df['label'].unique().tolist()
labels

['урлаг соёл',
 'эдийн засаг',
 'эрүүл мэнд',
 'хууль',
 'улс төр',
 'спорт',
 'технологи',
 'боловсрол',
 'байгал орчин']

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=999, stratify=df['label'])

In [13]:
DATA_COLUMN  = 'news'
LABEL_COLUMN = 'label'
label_list   = labels

In [14]:
train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)
test_InputExamples  = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)

In [15]:
LEARNING_RATE          = 2e-5
NUM_TRAIN_EPOCHS       = 20.0
WARMUP_PROPORTION      = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS     = 100
TRAIN_BATCH_SIZE       = 1
EVAL_BATCH_SIZE        = 1
PREDICT_BATCH_SIZE     = 1
ITERATIONS_PER_LOOP    = 100
NUM_TPU_CORES          = 8
MAX_SEQ_LENGTH         = LEN

should_train           = True

In [20]:
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features  = run_classifier.convert_examples_to_features(test_InputExamples , label_list, MAX_SEQ_LENGTH, tokenizer)

num_train_steps  = int(len(train_features) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_test_steps   = int(len(test_features) / EVAL_BATCH_SIZE)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

INFO:tensorflow:Writing example 0 of 68094
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Засгийн ▁газрын ▁гишүүн , ▁УИХ - ын ▁гишүүн ▁Х . Баттулга ▁сайд аасаа ▁тат зал зах ▁хүсэлтээ ▁өнгөрсөн ▁долоо ▁хоногт ▁өгсөн ▁билээ . ▁Тэрээр ▁сайд аасаа ▁татгалзах ▁хүсэлт дээ , ▁Засгийн ▁газрын ▁гишүүний ▁ажлаа ▁өгч , ▁УИХ - ын ▁гишүүний ▁үүргээ ▁үргэлж үүлэх ээр ▁болсноо ▁дурьдсан ▁бөгөөд ▁цаашид ▁үйлдвэрлэл , ▁хөдөө ▁аж ▁ахуйн ▁салбарт ▁эхлүүлсэн ▁ажлаа ▁үргэлж үүлэхэд ▁анхаарч ▁ажиллана ▁гэдгээ ▁мэдэгдсэн . ▁Өнгөрсөн ▁долоо ▁хоногт ▁Ерөнхийлөгчөөс ▁өргөн ▁барьсан ▁Засгийн ▁газрын ▁гишүүний ▁үүргийг ▁давхар ▁хашдаг ▁УИХ - ын ▁гишүүдийг ▁" дан ▁дээлтэй " ▁болгох ▁хуулийн ▁төслийг ▁гурван ▁сарын ▁хугацаанд ▁хэлэлцэж , ▁унагасан . ▁УИХ - ын ▁гишүүдийг ▁" давхар ▁дээл "- ийг ▁тайлах ▁хуулийн ▁төслийг ▁2014 ▁оноос ▁хэрэгжүүлэх ▁ёстой ▁гэдэг ▁дээр ▁Х . Баттулга ▁сайд ▁тууштай ▁байсан ▁бөгөөд ▁өгсөн ▁саналаа ▁бататгаж , ▁өргөдлөө ▁өгсөн ▁гэж ▁зарим ▁хүмүүс ▁

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:label: улс төр (id = 4)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Байгаль ▁орчны ▁салбарын ▁голлох ▁суурь ▁хуулиуд ▁1990 ▁оны ▁эхээр ▁батлагдаж , ▁түүнээс ▁хойшхи ▁хугацаанд ▁нэмэлт , ▁өөрчлөлтүүд ▁оруулж ▁ирсэн . ▁Одоогоор ▁мөрдөгдөж ▁буй ▁30 ▁гаруй ▁хуулиар ▁байгаль ▁орчин ▁хамгааллын ▁бүх ▁хариуцлагыг ▁төр ▁дангаараа ▁үүрч ▁байгаа . ▁Энэ ▁зохицуулалтыг ▁өөрчилж , ▁бохирдуулагч ▁нь ▁төлбөр ▁төлөгч ▁байх , ▁ашиг лагч ▁нь ▁хамгаалагч ▁байх ▁суурь ▁зарчимд ▁үндэслэн ▁салбарын ▁хуулиа ▁багц аар ▁нь ▁шинэчлэх ▁ажлыг ▁БОАЖЯ ▁2008 ▁оноос ▁хойш ▁хийсэн ▁юм . ▁Байгаль ▁орчны ▁багц ▁хуулийг ▁сайд ▁Л . Гансүх ▁2011 .12. 30- ны ▁өдөр ▁УИХ - ын ▁дарга ▁Д . Дэмбэрэл д ▁өргөн ▁барьсан ▁байна . ▁Г . Туяа [SEP]
INFO:tensorflow:input_ids: 4 1575 1248 444 7055 787 6207 2801 41 2322 9488 8 1585 9206 185 541 8 9248 485 176 7 1623 7744 60 476 161 1043 922 480 2042 168 3199 288 5477 8056 15 7 26 4040 2650 8 19492 9 1191 6880 38 8 508 4356 9 36

INFO:tensorflow:label: спорт (id = 5)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Энэ ▁сарын ▁2-3 - ны ▁өдрүүдэд ▁ “ Ж уул чны ▁аюулгүй ▁байдал ▁ба ▁аялал ▁жуулчлалын ▁эрсдлийн ▁менежмент ” ▁сургалт ▁болно . ▁Тус ▁сургалтыг ▁Нийслэлийн ▁Онцгой ▁байдлын ▁ерөнхий ▁газар , ▁Зооноз ын ▁халдварт ▁өвчин ▁судлалын ▁төвтэй ▁хамтран ▁зохион ▁байгуулахаар ▁болсон ▁байна . ▁Сургалтад ▁аялал ▁жуулчлалын ▁үйлчилгээний ▁байгууллагууд ▁болох ▁зочид ▁буудал , ▁дэн ▁буудал , ▁жуулчны ▁баазын ▁менежерүүд ▁хамрагдах ▁юм ▁байна . ▁Н . БАЙГАЛ Ь [SEP]
INFO:tensorflow:input_ids: 4 26 46 3310 12 50 1032 10 0 1392 2538 6769 400 205 240 839 1402 16063 4955 0 1062 57 7 365 4445 296 1246 447 339 48 8 28733 17 2577 945 2378 5934 243 101 2109 51 11 7 20166 839 1402 689 714 72 1490 2080 8 12945 2080 8 11563 17308 21093 2060 16 11 7 150 7 29934 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:input_ids: 4 2452 151 3727 870 594 113 12105 21 1080 1085 28312 30249 126 21190 94 2401 28562 2401 12 1392 299 7515 2401 12 621 5136 14009 20913 290 2326 10 7 26 1779 9 202 9165 103 387 311 224 431 111 515 870 5130 1302 1714 16 10 7 1045 8 927 8 1828 8 1012 384 4221 112 474 394 10621 27 2182 7512 16287 4263 8 10621 17223 4263 8 10621 163 770 4263 8 717 1069 13828 7530 8 8120 10637 9411 21 4263 8 13995 1069 13828 7530 8 23840 8 202 1612 163 770 8 30739 9957 16511 1732 10419 4263 8 3676 4460 9220 36 21688 8 1797 19428 7911 1311 4263 116 8785 7 491 2002 2366 2952 158 2004 314 52 7 990 1302 4434 12105 21 1085 1779 944 9 1151 2062 16 11 7 623 3521 144 44 325 193 7 7987 6770 12105 21 1085 28312 30249 126 21190 94 2401 28562 2401 12 1392 299 7515 2401 12 621 21190 94 3251 2860 361 144 701 2517 443 2452 151 30654 3659 228 785 3726 4255 3408 162 566 965 4616 256 4859 567 1662 267 14886 2473 16500 11014 151 1795 1945 285 11 7 12692 2952 594 18 1336 1014 7526 9 2615 1435 829 89 15

In [21]:
train_input_fn = run_classifier.input_fn_builder(
    features       = train_features,
    seq_length     = MAX_SEQ_LENGTH,
    is_training    = True,
    drop_remainder = True)

In [22]:
run_config = tf.contrib.tpu.RunConfig(
    model_dir = OUTPUT_DIR,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS,
    tpu_config = tf.contrib.tpu.TPUConfig(
        iterations_per_loop         = ITERATIONS_PER_LOOP,
        num_shards                  = NUM_TPU_CORES,
        per_host_input_for_training = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

model_fn = run_classifier.model_fn_builder(
    bert_config      = modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels       = len(label_list),
    init_checkpoint  = INIT_CHECKPOINT,
    learning_rate    = LEARNING_RATE,
    num_train_steps  = num_train_steps,
    num_warmup_steps = num_warmup_steps,
    use_tpu          = False,
    use_one_hot_embeddings = True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu            = False,
    eval_on_tpu        = False,
    model_fn           = model_fn,
    config             = run_config,
    train_batch_size   = TRAIN_BATCH_SIZE,
    eval_batch_size    = EVAL_BATCH_SIZE,
    predict_batch_size = PREDICT_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': './eduge_512_cased', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1cb1618f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, inp

In [23]:
from datetime import datetime

if should_train:
  print("Beginning Training!")
  current_time = datetime.now()
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (8, 512)
INFO:tensorflow:  name = input_mask, shape = (8, 512)
INFO:tensorflow:  name = label_ids, shape = (8,)
INFO:tensorflow:  name = segment_ids, shape = (8, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  n

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

MemoryError: 